In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/FundDataWithMonthlyPrices_v3.csv')
data.head()

,fundRUN,fundName,fundSeries,aafmCategory,svsCategory,svsCategoryId,currency,fundRUNSeries,userCategory,1/31/15,...,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21
0,8812-9,A. CHILE CALIFICADO,D,Inversionistas Calificados Accionario Nacional,FM DIRIGIDO A INVERSIONISTAS CALIFICADOS,8.0,P,8812-9D,"Domestic Bond, < 365",767.0540,...,706.7567,665.9628,712.7872,727.2119,690.8105,659.9857,636.7984,728.8420,764.2218,769.6162
1,8289-9,ACCIONES CHILE,APV,Accionario Nacional Large CAP,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8289-9APV,Domestic Equity,857.0135,...,686.6956,629.7418,675.7632,688.7364,652.1686,629.9546,607.5724,684.7373,710.2978,725.5830
2,8076-4,ACCIONES CHILENAS,APV,Accionario Nacional Large CAP,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8076-4APV,Domestic Equity,4264.7434,...,4026.5817,3646.4398,3938.0199,3999.2216,3752.6015,3618.8839,3515.6612,3980.6780,4105.8619,4205.8988
3,9254-1,ACCIONES EUROPA,ALTO,Accionario Europa Desarrollado,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,9254-1ALTO,International Equity,NaN,...,911.2628,932.6800,977.8669,926.3185,992.1376,960.9146,872.7992,1041.4013,1006.0475,1025.7878
4,8247-3,ACCIONES LATAM,A,Accionario America Latina,FM DE INVERSION EN INSTRUMENTOS DE CAPITALIZACION,5.0,P,8247-3A,International Equity,1693.6139,...,1397.8727,1480.2771,1610.8085,1663.5947,1604.6280,1499.6090,1470.9928,1785.2915,1797.7134,1767.1439


In [4]:
ts = data.iloc[:, 9:].T
ts.index = pd.to_datetime(ts.index)
ts.head()

,0,1,2,3,4,5,6,7,8,9,...,301,302,303,304,305,306,307,308,309,310
2015-01-31,767.0540,857.0135,4264.7434,NaN,1693.6139,NaN,2871.3193,1268.8118,1101.4274,1144.2227,...,NaN,1608.6475,1675.3323,1008.7618,2399.8201,1634.0381,NaN,28476.9327,1376.8601,1600.2140
2015-02-28,788.5753,882.9958,4467.0361,NaN,1722.7516,NaN,2966.3837,1290.0886,1115.8103,1153.7850,...,NaN,1652.8484,1718.5874,1034.1400,2417.0463,1640.8835,NaN,28546.6508,1390.0569,1649.5923
2015-03-31,785.9268,873.0740,4386.0161,NaN,1635.1843,NaN,2937.4799,1289.0306,1117.2210,1153.3668,...,NaN,1683.9404,1710.4401,1032.9181,2435.3677,1643.6302,NaN,28621.2591,1399.5459,1652.2255
2015-04-30,797.9719,882.3608,4499.4617,NaN,1717.7214,NaN,2999.4534,1277.8355,1102.7034,1139.2270,...,NaN,1626.8025,1674.4244,1008.6092,2427.1349,1647.2129,NaN,28697.0684,1386.3464,1633.9630
2015-05-31,792.9950,874.7575,4429.9785,NaN,1632.6834,NaN,2978.0368,1283.8804,1107.4667,1146.2572,...,NaN,1659.8826,1703.9052,1026.8735,2442.1070,1653.3580,NaN,28776.6986,1392.5871,1659.2681


In [5]:
def get_cagr(series):
    series = series.dropna()
    start = series[0]
    end = series[-1]
    n = len(series)
    return (end/start) ** (1/(n/12)) - 1

In [18]:
def check_series(series):
    return (len(series) - series.isna().sum()) < 6

In [19]:
def get_ann_return(series):
    if check_series(series):
        return np.nan
    return get_cagr(series)

In [20]:
def get_ann_std(series):
    if check_series(series):
        return np.nan
    series = series.dropna()
    return series.pct_change().std() * np.sqrt(12)

In [44]:
def geo_mean(series):
    n = len(series)
    arr = np.ones(n)
    return np.product(series + arr) ** (1/n) - 1

In [37]:
returns = {}
years = [*range(2015, 2021)]
for year in years:
    df = ts.loc[str(year), :]
    returns[f'{str(year)}_return'] = df.apply(lambda x: get_ann_return(x))
    returns[f'{str(year)}_stdev'] = df.apply(lambda x: get_ann_std(x))

In [38]:
returns = pd.DataFrame(returns)
returns

,2015_return,2015_stdev,2016_return,2016_stdev,2017_return,2017_stdev,2018_return,2018_stdev,2019_return,2019_stdev,2020_return,2020_stdev
0,-0.054205,0.091659,0.026891,0.085787,0.358370,0.236755,-0.144572,0.093647,-0.164147,0.174702,-0.034894,0.310821
1,-0.076368,0.094678,0.006092,0.104716,0.264741,0.225013,-0.137664,0.102476,-0.149665,0.145456,-0.117688,0.331981
2,-0.067128,0.104892,0.179741,0.135288,0.274556,0.209719,-0.150732,0.099343,-0.159611,0.126905,-0.119877,0.326454
3,NaN,NaN,-0.028956,0.228325,0.166793,0.131747,-0.186715,0.153160,0.184751,0.146168,-0.105197,0.357608
4,-0.144873,0.159978,0.205651,0.180984,0.115831,0.082301,-0.113595,0.198663,0.131331,0.154057,-0.226073,0.490279
...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.029391,0.005130,0.034709,0.006408,0.022352,0.004486,0.019793,0.002763,0.029459,0.009730,0.019787,0.009732
307,NaN,NaN,0.020086,0.012903,0.018350,0.009322,0.018391,0.007614,0.029939,0.012158,0.025729,0.014162
308,0.029558,0.000342,0.033427,0.000928,0.025904,0.001381,0.023335,0.000471,0.023649,0.001802,0.009376,0.003654
309,0.022890,0.022591,0.049729,0.038484,0.004787,0.020287,0.035443,0.011174,0.000783,0.092870,0.007396,0.043954


In [49]:
cols = [x for x in returns.columns if 'return' in x]
returns['avg_ann_return'] = returns[cols].apply(lambda x: geo_mean(x), axis=1)
returns['ann_stdev'] = returns[cols].std(axis=1)
returns

,2015_return,2015_stdev,2016_return,2016_stdev,2017_return,2017_stdev,2018_return,2018_stdev,2019_return,2019_stdev,2020_return,2020_stdev,avg_ann_return,ann_stdev
0,-0.054205,0.091659,0.026891,0.085787,0.358370,0.236755,-0.144572,0.093647,-0.164147,0.174702,-0.034894,0.310821,-0.015525,0.190330
1,-0.076368,0.094678,0.006092,0.104716,0.264741,0.225013,-0.137664,0.102476,-0.149665,0.145456,-0.117688,0.331981,-0.044631,0.157334
2,-0.067128,0.104892,0.179741,0.135288,0.274556,0.209719,-0.150732,0.099343,-0.159611,0.126905,-0.119877,0.326454,-0.020873,0.186789
3,NaN,NaN,-0.028956,0.228325,0.166793,0.131747,-0.186715,0.153160,0.184751,0.146168,-0.105197,0.357608,-0.003895,0.164721
4,-0.144873,0.159978,0.205651,0.180984,0.115831,0.082301,-0.113595,0.198663,0.131331,0.154057,-0.226073,0.490279,-0.018715,0.177646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.029391,0.005130,0.034709,0.006408,0.022352,0.004486,0.019793,0.002763,0.029459,0.009730,0.019787,0.009732,0.025900,0.006160
307,NaN,NaN,0.020086,0.012903,0.018350,0.009322,0.018391,0.007614,0.029939,0.012158,0.025729,0.014162,0.018706,0.005141
308,0.029558,0.000342,0.033427,0.000928,0.025904,0.001381,0.023335,0.000471,0.023649,0.001802,0.009376,0.003654,0.024181,0.008217
309,0.022890,0.022591,0.049729,0.038484,0.004787,0.020287,0.035443,0.011174,0.000783,0.092870,0.007396,0.043954,0.020018,0.019442


In [51]:
fund_data = data.iloc[:, :9]
pd.concat([fund_data, returns*100], axis=1).to_csv('../data/FundDataAnnualReturns.csv', index=False)

<hr>

In [60]:
current_dataset = pd.read_csv('../data/FundDataWithMonthlyReturnsTSNEScoredScaled.csv')
pd.concat(
    [current_dataset, returns.iloc[:, :-2] * 100], axis=1
).to_csv('../data/FundDataMonthlyReturnsTSNEScoredScaledwithAnnualReturns.csv', index=False)